# Graph Classification

## Graph Classes

we use the following graph classes:
- Chemical Graphs (Molecules)
- Random
- Small World
- Scale Free

## Setup

We use pyg (pytorch-geometric) to generate the model to train.
The model is a GCN with 2 layers and 32 hidden units.

In [10]:
import torch_geometric
import torch

print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.mps.is_available())
print(torch_geometric.__version__)

1.13.1
None
True
2.2.0


In [40]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx

# we need to read the dataset from the pickle file
import pickle

with open('dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

for key in dataset.keys():
    print(f"datasetkey: {key}, shape: {len(dataset[key])}")

# we need to convert the networkx graphs to pytorch geometric graphs

class_key = {
    'random': 0,
    'smallworld': 1,
    'scalefree': 2,
}
pyg_dataset = []
for key in dataset.keys():
    for i in range(len(dataset[key])):
        graph = dataset[key][i]
        graph_tensor = from_networkx(graph)
        graph_tensor.y = torch.tensor([class_key[key]])
        pyg_dataset.append(graph_tensor)

datasetkey: random, shape: 95
datasetkey: smallworld, shape: 95
datasetkey: scalefree, shape: 95


In [1]:
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx
import pickle


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']


    def process(self):
        # Read data into huge `Data` list.
        with open('dataset.pickle', 'rb') as f:
            dataset = pickle.load(f)

        class_key = dict(zip(dataset.keys(), range(len(dataset.keys()))))
        print(class_key)
        data_list = []
        for key in dataset.keys():
            for i in range(len(dataset[key])):
                graph = dataset[key][i]
                graph_tensor = from_networkx(graph, group_node_attrs=["label", "betweenness", "degree"])
                graph_tensor.y = torch.tensor([class_key[key]])
                # graph_tensor.x = graph_tensor.x.view(-1, 1)
                # graph_tensor.x = graph_tensor.x.to(torch.float)
                data_list.append(graph_tensor)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [4]:
import torch
from torch_geometric.datasets import TUDataset

dataset = MyOwnDataset(root='data/CustomData')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print(data.x)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...


{'random': 0, 'smallworld': 1, 'scalefree': 2, 'complete': 3, 'line': 4, 'tree': 5}

Dataset: MyOwnDataset(864):
Number of graphs: 864
Number of features: 3
Number of classes: 6

Data(edge_index=[2, 10], x=[5, 3], y=[1])
tensor([[0.0000, 0.1667, 0.5000],
        [1.0000, 0.1667, 0.5000],
        [2.0000, 0.1667, 0.5000],
        [3.0000, 0.1667, 0.5000],
        [4.0000, 0.1667, 0.5000]])
Number of nodes: 5
Number of edges: 10
Average node degree: 2.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


In [3]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:600]
test_dataset = dataset[600:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 864
Number of test graphs: 0


In [56]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 63730], x=[2991, 3], y=[64], batch=[2991], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 85752], x=[3441, 3], y=[64], batch=[3441], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 102210], x=[3826, 3], y=[64], batch=[3826], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 71108], x=[3143, 3], y=[64], batch=[3143], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 60772], x=[2997, 3], y=[64], batch=[2997], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 81852], x=[3264, 3], y=[64], batch=[3264], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 92530], x=[3674, 3], y=[64], batch=[3674], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 65164], x=[3356, 3], 

In [57]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(3, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=3, bias=True)
)


In [48]:
from torch_geometric.nn import GATConv, global_mean_pool


# Create your GNN model:
class GNN(torch.nn.Module):
    def __init__(self):
        self.conv1 = GATConv(dataset.num_features, 16)
        self.conv2 = GATConv(16, 16)
        self.lin = Linear(16, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
         # Choose between different GNN building blocks:
        x = global_mean_pool(x, batch)
        return self.lin(x)


In [58]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
# model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


confusion_matrix = torch.zeros(3, 3)
for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.6635, Test Acc: 0.6824
Epoch: 002, Train Acc: 0.6667, Test Acc: 0.6436
Epoch: 003, Train Acc: 0.6673, Test Acc: 0.6436
Epoch: 004, Train Acc: 0.6670, Test Acc: 0.6436
Epoch: 005, Train Acc: 0.6638, Test Acc: 0.6812
Epoch: 006, Train Acc: 0.6673, Test Acc: 0.6436
Epoch: 007, Train Acc: 0.6713, Test Acc: 0.6703
Epoch: 008, Train Acc: 0.6622, Test Acc: 0.6824
Epoch: 009, Train Acc: 0.6614, Test Acc: 0.6812
Epoch: 010, Train Acc: 0.6643, Test Acc: 0.6824
Epoch: 011, Train Acc: 0.6617, Test Acc: 0.6788
Epoch: 012, Train Acc: 0.6722, Test Acc: 0.6473
Epoch: 013, Train Acc: 0.6694, Test Acc: 0.6558
Epoch: 014, Train Acc: 0.6673, Test Acc: 0.6448
Epoch: 015, Train Acc: 0.6716, Test Acc: 0.7176
Epoch: 016, Train Acc: 0.6630, Test Acc: 0.6824
Epoch: 017, Train Acc: 0.6778, Test Acc: 0.6982
Epoch: 018, Train Acc: 0.6624, Test Acc: 0.6788
Epoch: 019, Train Acc: 0.6643, Test Acc: 0.6824
Epoch: 020, Train Acc: 0.6673, Test Acc: 0.6436
Epoch: 021, Train Acc: 0.6705, Test Acc: